In [1]:
# crearemos un generador de folletos a partir de ;la pgina web de una empresa
#para ello ussaremos web scrapping y llms 


#importamos la librerias
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

In [2]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

OpenAI API Key existe y empieza por sk-proj-
Google API Key existe y empieza por AIzaSyAh


In [4]:
# Conéctese a OpenAI y Google

openai = OpenAI()


In [5]:
# Un mensaje genérico del sistema

system_message = "Eres un asistente útil"

In [6]:

#en esta clase haremos el web scrapping 
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No se ha encontrado título de la página"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [7]:
#creamos el mensaje de sismtema

system_message = "Eres un asistente que analiza el contenido de la página de inicio \
del sitio web de una empresa y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados.\
Responde en formato Markdown."

In [8]:
#creamos la funcion para hacer la consulta con openAI

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:  #mostrar la respuesta en stream
        result += chunk.choices[0].delta.content or ""
        yield result

In [15]:
#creamos funcion para hacer consulta a google gemini
genai.configure(api_key=google_api_key)
def stream_gemini(prompt):
    

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt, stream=True)
    result = ""
    for chunk in response:
        result += chunk.text or ""
        yield result

In [16]:
def stream_brochure(company_name, url, model):
    prompt = f"Genera un folleto de la empresa {company_name}. Esta es su página de destino:\n"    
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [17]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa:"),
        gr.Textbox(label="Landing page, recuerda incluir http:// o https://"),
        gr.Dropdown(["GPT", "Gemini"], label="Selecciona un modelo")],
    outputs=[gr.Markdown(label="Folleto:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [18]:
#Anadamos ollama a nuestro programa para poder tener un llm que se ejecute de forma local 

import ollama

MODEL = "gemma3"

In [26]:
def stream_llama(prompt):
    
    mensaje=  [{"role": "user", "content": prompt}]
    response = ollama.chat(model=MODEL, messages=mensaje,stream= True)

    result = ""
    for chunk in response:
        content = chunk.get("message", {}).get("content", "")
        result += content
        yield result
    

In [29]:
def stream_brochure(company_name, url, model):
    prompt = f"Genera un folleto de la empresa {company_name}. Esta es su página de destino:\n"    
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    elif model== "Ollama":
        result = stream_llama(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [30]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa:"),
        gr.Textbox(label="Landing page, recuerda incluir http:// o https://"),
        gr.Dropdown(["GPT", "Gemini","Ollama"], label="Selecciona un modelo")],
    outputs=[gr.Markdown(label="Folleto:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
